# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_data = pd.read_csv("../WeatherPy/output_data/cities.csv")


del cities_data['Unnamed: 0']


cities_data.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,barrow,71.2906,-156.7887,-5.78,77,0,14.97,US,1649961022
1,ushuaia,-54.8000,-68.3000,53.22,54,20,13.80,AR,1649960870
2,tasiilaq,65.6145,-37.6368,32.16,100,100,2.84,GL,1649961085
3,busselton,-33.6500,115.3333,57.60,84,0,8.08,AU,1649961100
4,pevek,69.7008,170.3133,6.26,92,100,0.40,RU,1649961100


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key) 

In [4]:
# Bring in location data pairs, humidity data, define map and layer, and add layer to map
locations = cities_data[["Lat", "Lng"]]
humidity = cities_data["Humidity"]

fig = gmaps.figure(center = (20,0), zoom_level=2.2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=350,
                                 point_radius=5)
fig.add_layer(heat_layer)
fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
clouds = cities_data['Cloudiness']
temp = cities_data['Max Temp']
wind = cities_data['Wind Speed']

ideal_data = cities_data.loc[(clouds < 10)]
ideal_data = ideal_data.loc[(temp > 75) & (temp < 85)]
ideal_data = ideal_data.loc[(wind < 10)]
ideal_data.dropna()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
87,ascension,30.2169,-90.9001,81.07,57,0,8.05,US,1649961130
223,nizwa,22.9333,57.5333,84.56,25,7,4.45,OM,1649961155
226,nanakuli,21.3906,-158.1547,78.75,64,0,9.22,US,1649961183
319,grand gaube,-20.0064,57.6608,79.38,77,3,8.99,MU,1649961221
332,vallenar,-28.5708,-70.7581,76.30,29,0,9.08,CL,1649961226
380,gatesville,31.4352,-97.7439,75.15,30,0,5.75,US,1649961244
384,lagos,6.5833,3.7500,84.52,75,6,7.72,NG,1649961080
412,dehloran,32.6941,47.2679,79.65,16,8,4.00,IR,1649961256
440,santiago del estero,-27.7951,-64.2615,75.13,32,0,7.43,AR,1649961265
457,caravelas,-17.7125,-39.2481,84.43,68,6,9.06,BR,1649961273


### Hotel Map
* Create a dataframe named `hotel_df` that is the df created above only with the pertinent data.  See the prepared code below (`info_box_template`) to determine what data you will need for your final map.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels (aka "lodging") within 5000 meters.
* Utilize the Google Places API to find the first hotel search result.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create dataframe and empty column
ideal_data['Hotel Name'] = ''
ideal_data['Hotel Address'] = ''
hotel_df = ideal_data[['City', 'Country','Hotel Name', 'Hotel Address',
                     'Humidity', 'Lat', 'Lng', 'Max Temp', 'Wind Speed']]

params = {"radius": 5000,"keyword": "hotel","key": g_key}

In [7]:
# Loop through dataframe and add nearby hotel result
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    name_address = requests.get(base_url, params=params)
    
    name_address = name_address.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        hotel_df.loc[index, "Hotel Address"] = name_address["results"][0]["vicinity"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


/opt/anaconda3/envs/PythonData38/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Missing field/result... skipping.
Missing field/result... skipping.


In [8]:
hotel_df.loc[hotel_df['Hotel Name'] != '']

,City,Country,Hotel Name,Hotel Address,Humidity,Lat,Lng,Max Temp,Wind Speed
87,ascension,US,Comfort Suites Near Tanger Outlet Mall,"2821 Cabela's Pkwy, Gonzales",57,30.2169,-90.9001,81.07,8.05
223,nizwa,OM,Al Diyar Hotel,Street 21 Nizwa OM,25,22.9333,57.5333,84.56,4.45
319,grand gaube,MU,Zilwa Attitude Hotel,"Royal Road, Calodyne",77,-20.0064,57.6608,79.38,8.99
332,vallenar,CL,Hotel Origenes,Llanos de Ferrera 12 Vallenar,29,-28.5708,-70.7581,76.30,9.08
380,gatesville,US,Holiday Inn Express & Suites Gatesville - N. F...,"2904 TX-36, Gatesville",30,31.4352,-97.7439,75.15,5.75
412,dehloran,IR,هتل سعید,"M7PF+2G6, Dehloran",16,32.6941,47.2679,79.65,4.00
440,santiago del estero,AR,Hotel Libertador,"Catamarca 47, DPA",32,-27.7951,-64.2615,75.13,7.43
457,caravelas,BR,Pousada dos Navegantes,"R. das Palmeiras, 45 - Caravelas",68,-17.7125,-39.2481,84.43,9.06
473,yulara,AU,Outback Pioneer Hotel & Lodge - Ayers Rock Resort,"2 Yulara Dr, Yulara",31,-25.2406,130.9889,75.04,5.75
539,rafaela,AR,Hotel Plaza Rafaela,"FNF Rafaela Santa Fe AR, Blvd. G. Lehmann 380",18,-31.2503,-61.4867,75.99,3.00


In [9]:
# NOTE: Do not change any of the code in this cell
# Analyze the code below for variables but these should be all columns in hotel_df

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))